Develop an advanced support ticket categorization system that accurately classifies incoming tickets, assigns relevant tags based on their content, implements mechanisms and generate the first response based on the sentiment for prioritizing tickets for prompt resolution.


## **Installing and Importing Necessary Libraries and Dependencies**

In [ ]:
# for loading and manipulating data.
# try:
#   import pandas as pd
# except:
#   pip uninstall numpy
#   pip install numpy==1.15.1
#   import pandas as pd

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
# # for time computations.
import time

In [ ]:
# Installation for GPU llama-cpp-python.
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.85 --force-reinstall --no-cache-dir -q

In [ ]:
# pip install llama-cpp-python==0.1.85 --force-reinstall --no-cache-dir -q

In [ ]:
# Importing the Llama class from the llama_cpp module.
from llama_cpp import Llama

In [ ]:
# For downloading the models from HF Hub.
# !pip install huggingface_hub==0.20.3 pandas==1.5.3 -q

In [ ]:
# Function to download the model from the Hugging Face model hub.
from huggingface_hub import hf_hub_download

# Importing the json module.
import json

In [ ]:
import pandas as pd

## **Loading the Data**

In [ ]:
# Loading the data into df
df = pd.read_csv("Support_ticket_text_data_mid_term.csv")

# Creating copy of 'df' in the variable data
data = df.copy()

## **Data Overview**

### Checking the first 5 rows of the data

In [ ]:
# first 5 rows of the data
data.head(5)

### Checking the shape of the data

In [ ]:
# shape of data
data.shape

In [ ]:
# There are 21 rows and 2 columns present in this data.

### Checking the missing values in the data

In [ ]:
# Missing values in data
data.isna().sum().sum()

In [ ]:
# From the above output we identify there are no missing values in the dataset.

## **Model Building**

### Loading the model

In [ ]:
# model name and model base name
model_name_or_path = "TheBloke/Mistral-7B-Instruct-v0.2-GGUF"
model_basename = "mistral-7b-instruct-v0.2.Q6_K.gguf"

In [ ]:
# Declaring repo_id and filename
model_path = hf_hub_download(
    repo_id=model_name_or_path, # repo_id = model_name_or_path
    filename=model_basename # filename = model_basename
)

In [ ]:
# Defining the llm model - Llama (Run using GPU)

llm = Llama(
    model_path=model_path,
    n_ctx=1024, # Context window
)

### Utility functions

In [ ]:
# defining a function to parse the JSON output from the model
def extract_json_data(json_str):
    try:
        # Find the indices of the opening and closing curly braces
        json_start = json_str.find('{')
        json_end = json_str.rfind('}')

        if json_start != -1 and json_end != -1:
            extracted_category = json_str[json_start:json_end + 1]  # Extract the JSON object
            data_dict = json.loads(extracted_category)
            return data_dict
        else:
            print(f"Warning: JSON object not found in response: {json_str}")
            return {}
    except json.JSONDecodeError as e:
        print(f"Error parsing JSON: {e}")
        return {}

## **Task 1: Ticket Categorization and Returning Structured Output**

In [ ]:
# creating a copy of the data
data_1 = data.copy()

In [ ]:
# Defining the response funciton for Task 1.
def response_1(prompt,ticket):
    model_output = llm(
      f"""
      Q: {prompt}
      Support ticket: {ticket}
      A:
      """,
      max_tokens=10, # defining the maximum number of tokens the model should generate for this task.
      stop=["Q:", "\n"],
      temperature=0.01, # temperature set to 0.01(low) for deterministic output.
      echo=False,
    )

    temp_output = model_output["choices"][0]["text"]
    final_output = temp_output[temp_output.index('{'):]

    return final_output

In [ ]:
# Prompt creation for task 1
prompt_1 = """
   As an AI, your job is to categorize IT support tickets. 
   Please label each ticket as either a Hardware Issue, Data Recovery, or Technical Issue. 
   Your response should be in the format: {"category": "Hardware Issues"}, {"category": "Data Recovery"}, or {"category": "Technical Issues"}. 
   Keep your output simple and accurate. Ensure that all curly braces are closed and there are no additional characters in the output.
"""

**Note**: The output of the model should be in a structured format (JSON format).

In [ ]:
# Utilizing generate_llama_response as a function on the variable: support_ticket_text 
start = time.time()
data_1['model_response'] = data_1['support_ticket_text'].apply(lambda x: response_1(prompt_1, x))
end = time.time()

In [ ]:
# Time taken for model to return output
print("Time taken:", round((end-start)),"seconds")

In [ ]:
# Initial model output
data_1['model_response'].head(5)

In [ ]:
# Displaying the support ticket text
i = 6
print(data_1.loc[i,'support_ticket_text'])

In [ ]:
# Model output
print(data_1.loc[i, 'model_response'])

In [ ]:
# Applying the function to the model response
data_1['model_response_parsed'] = data_1['model_response'].apply(extract_json_data)
data_1['model_response_parsed'].head()

In [ ]:
# Model output after extracting JSON data
data_1['model_response_parsed'].value_counts()

In [ ]:
# Normalizing the model_response_parsed column
model_response_parsed_df_1 = pd.json_normalize(data_1['model_response_parsed'])
model_response_parsed_df_1.head()

In [ ]:
# Concatinating two dataframes
data_with_parsed_model_output_1 = pd.concat([data_1, model_response_parsed_df_1], axis=1)
data_with_parsed_model_output_1.head()

In [ ]:
# Dropping model_response and model_response_parsed columns
final_data_1 = data_with_parsed_model_output_1.drop(['model_response','model_response_parsed'], axis=1)
final_data_1.head()

## **Task 2: Creating Tags**

In [ ]:
# creating a copy of the data
data_2 = data.copy()

In [ ]:
def response_2(prompt,ticket,category):
    model_output = llm(
      f"""
      Q: {prompt}
      Support ticket: {ticket}
      Category: {category}
      A:
      """,
      max_tokens=1024,  # defining the maximum number of tokens the model should generate for this task.
      stop=["Q:", "\n"],
      temperature=0.01,  # temperature set to 0.01(low) for deterministic output.
      echo=False,
    )

    temp_output = model_output["choices"][0]["text"]
    final_output = temp_output[temp_output.index('{'):]

    return final_output

In [ ]:
# Prompt creation for task 2
prompt_2 = """
   As an AI, your task is to label IT support tickets with relevant tags. 
   Please identify the most appropriate keywords and include them in your response. 
   Your output should be formatted as follows: {"tags": ["Wifi", "Data Loss", "Connection Issues", "Battery"]}.
   Keep your output simple and accurate. Ensure that all curly braces are closed and there are no additional characters in the output.
"""

**Note**: The output of the model should be in a structured format (JSON format).

In [ ]:
# Utilizing generate_llama_response as a function on the variable: support_ticket_text
start = time.time()
data_2["model_response"]=final_data_1[['support_ticket_text','category']].apply(lambda x: response_2(prompt_2, x[0],x[1]),axis =1)
end = time.time()

In [ ]:
# Time taken for model to generate output
print("Time taken:",round((end-start))," seconds")

In [ ]:
# Initial model output
data_2['model_response'].head(5)

In [ ]:
# Support ticket text
i = 0
print(data_2.loc[i,'support_ticket_text'])

In [ ]:
# Model output
print(data_2.loc[i,'model_response'])

In [ ]:
# Applying the function to the model response
data_2['model_response_parsed'] = data_2['model_response'].apply(extract_json_data)

In [ ]:
# Model output after extracting JSON data
data_2["model_response_parsed"]

In [ ]:
# Normalizing the model_response_parsed column
model_response_parsed_df_2 = pd.json_normalize(data_2['model_response_parsed'])
model_response_parsed_df_2.head()

In [ ]:
# Concatinating two dataframes
data_with_parsed_model_output_2 = pd.concat([data_2, model_response_parsed_df_2], axis=1)
data_with_parsed_model_output_2.head()

In [ ]:
# Dropping model_response and model_response_parsed columns
final_data_2 = data_with_parsed_model_output_2.drop(['model_response','model_response_parsed'], axis=1)
final_data_2.head()

In [ ]:
# Checking the value counts of Category column
final_data_2['tags'].value_counts()

In [ ]:
# Concatinating two dataframes
final_data_2 = pd.concat([final_data_2,final_data_1["category"]],axis=1)

In [ ]:
# viewing newly updated dataframe
final_data_2 = final_data_2[["support_tick_id","support_ticket_text","category","tags"]]
final_data_2

## **Task 3: Assigning Priority and ETA**

In [ ]:
# creating a copy of the data
data_3 = data.copy()

In [ ]:
# Function created to generate an output from the model
def response_3(prompt,ticket,category,tags):
    model_output = llm(
      f"""
      Q: {prompt}
      Support ticket: {ticket}
      Category: {category}
      Tags: {tags}
      A:
      """,
      max_tokens=20,   # defining the maximum number of tokens the model should generate for this task.
      stop=["Q:", "\n"],
      temperature=0.01,  # temperature set to 0.01(low) for deterministic output.
      echo=False,
    )

    temp_output = model_output["choices"][0]["text"]
    final_output = temp_output[temp_output.index('{'):]

    return final_output

In [ ]:
# Prompt creation for task 3
prompt_3 = """
    As an AI, your task is to determine the priority and estimated time to resolve (ETA) for IT support tickets. 
    Consider the severity of the issue, the time needed for resolution, and customer satisfaction. 
    Your response should be in the format: {"priority": "High", "eta": "2 Days"}.
    Keep your output simple and accurate. Ensure that all curly braces are closed and there are no additional characters in the output.
"""

**Note**: The output of the model should be in a structured format (JSON format).

In [ ]:
# Utilizing generate_llama_response as a function on the variable: support_ticket_text  
start = time.time()
data_3['model_response'] = final_data_2[['support_ticket_text','category','tags']].apply(lambda x: response_3(prompt_3, x[0],x[1],x[2]),axis=1)
end = time.time()

In [ ]:
# Time taken for model to generate output
print("Time taken:",round((end-start))," seconds")

In [ ]:
# Initial model output
data_3['model_response'].head(5)

In [ ]:
# Support ticket text
i = 3
print(data_3.loc[i,'support_ticket_text'])

In [ ]:
# Model output
print(data_3.loc[i,'model_response'])

In [ ]:
# Applying the function to the model response
data_3['model_response_parsed'] = data_3['model_response'].apply(extract_json_data)
data_3['model_response_parsed'].head()

In [ ]:
# Normalizing the model_response_parsed column
model_response_parsed_df_3 = pd.json_normalize(data_3['model_response_parsed'])
model_response_parsed_df_3.head(21)

In [ ]:
# Concatinating two dataframes
data_with_parsed_model_output_3 = pd.concat([data_3, model_response_parsed_df_3], axis=1)
data_with_parsed_model_output_3.head()

In [ ]:
# Dropping model_response and model_response_parsed columns
final_data_3 = data_with_parsed_model_output_3.drop(['model_response','model_response_parsed'], axis=1)
final_data_3.head()

In [ ]:
# Concatinating two dataframes
final_data_3 = pd.concat([final_data_3,final_data_2[["category","tags"]]],axis=1)

In [ ]:
# Creating new dataframe
final_data_3 = final_data_3[["support_tick_id","support_ticket_text","category","tags","priority","eta"]]

In [ ]:
# viewing newly updated dataframe
final_data_3

## **Task 4 - Creating a Draft Response**

In [ ]:
# creating a copy of the data
data_4 = data.copy()

In [ ]:
# Function to generate output from the model
def response_4(prompt,ticket,category,tags,priority,eta):
    model_output = llm(
      f"""
      Q: {prompt}
      Support ticket: {ticket}
      Category : {category}
      Tags : {tags}
      Priority: {priority}
      ETA: {eta}
      A:
      """,
      max_tokens=1024,  # defining the maximum number of tokens the model should generate for this task.
      stop=["Q:", "\n"],
      temperature=0.01,  # temperature set to 0.01(low) for deterministic output.
      echo=False,
    )

    temp_output = model_output["choices"][0]["text"]

    return temp_output

In [ ]:
# Prompt creation for task 4
prompt_4 = """
    As an AI, your task is to draft a response for IT support tickets. 
    Consider customer satisfaction, the severity of the issue, and the company's responsibility. 
    Your response should be in the format: {"response": "This is a draft response"}. 
    Ensure your response is empathetic, professional, helpful, and concise.
    Please ensure that all curly braces are closed and there are no additional characters in the output.
"""

**Note** : For this task, we will not be using the *`extract_json_data`* function. Hence, the output from the model should be a plain string and not a JSON object.

In [ ]:
# Utilizing generate_llama_response as a function on the variable: support_ticket_text 
start = time.time()
data_4['model_response'] = final_data_3[['support_ticket_text','category','tags','priority','eta']].apply(lambda x: response_4(prompt_4, x[0],x[1],x[2],x[3],x[4]),axis=1)
end = time.time()

In [ ]:
# Time taken for output to be generated by model
print("Time taken:", round((end-start)),"seconds")

In [ ]:
# Initial model output
data_4['model_response'].head(21)

In [ ]:
# Support ticket text
i = 2
print(data_4.loc[i,'support_ticket_text'])

In [ ]:
# Model output
print(data_4.loc[i,'model_response'])

In [ ]:
# Applying the function to the model response
data_4['model_response_parsed'] = data_4['model_response'].apply(extract_json_data)
data_4['model_response_parsed'].head()

In [ ]:
# Normalizing the model_response_parsed column
model_response_parsed_df_4 = pd.json_normalize(data_4['model_response_parsed'])
model_response_parsed_df_4.head(21)

In [ ]:
# Concatinating two dataframes
final_data_4 = pd.concat([final_data_3,model_response_parsed_df_4],axis=1)

In [ ]:
# Renaming the dataframe
final_data_4.rename(columns={"model_response_parsed":"response"},inplace=True)

In [ ]:
# Viewing newly updated dataframe
final_data_4

## **Model Output Analysis**

In [ ]:
# Creating a copy of the dataframe of task 4
final_data = final_data_4.copy()

In [ ]:
# Value counts of category
final_data['category'].value_counts()

The model output for **category**:
> "Technical Issues" for 8 tickets

> "Hardware Issues" for 7 tickets

> "Data Recovery" for 6 tickets

In [ ]:
# Value counts of priority
final_data["priority"].value_counts()

The model output for **priority** of:

> "High" to 19 tickets

> "Medium" to 2 tickets

In [ ]:
# Value counts of ETA
final_data["eta"].value_counts()

The model output for **ETA** of:
> "3 Days" to 12 tickets

> "1 Day" to 9 tickets.

Let's dive in a bit deeper here.

In [ ]:
 # Group by data with regard to categories and ETA.
final_data.groupby(['category', 'eta']).support_tick_id.count()

> Most "Data Recovery" tickets are estimated by the model to be resolved in "3 Days".

> Most "Hardware Issues" tickets are estimated by the model to be resovled in "3 Days".

> Most "Technical Isses" tickets are estimated by the model to be resovled in "1 Day".

In [ ]:
# Final_data(output) generated by model.
final_data.head()

## **Actionable Insights and Recommendations**

**Insights:**
> A detailed company information in the prompts provide better model output.

> Adjust priority levels to align with your business's actual capabilities.

> Curating responses to a specific business by adjusting prompts or outputs.

> Adjust or expand categories to match your business's support needs. 

> Overall, The model's estimation of resolution times aligns with real-world scenarios.

**Recommendations:**
> Fine-tune the model with your company's data or profile for an improved performance.

> Adjust "priority" of support tickets to reflect priorities the business can actually facilitate.

> Need to evaluate on the format of responses with regard to the mail/response delivery methods.

> Require a thorough test of the model with actual data before implementation.